In [1]:
import os
import re
import numpy as np
import tensorflow as tf

# 文件路径
log_file = r"C:\data\result\txt\predictions_log.txt"
video_base_path = r"C:\data\video"

# 初始化特征和标签列表
features, labels = [], []

# 读取文件并解析数据
with open(log_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

i = 0
while i < len(lines) - 1:  # 遍历文件
    label_line = lines[i].strip()
    data_line = lines[i + 1].strip()
    
    # 提取关键信息（例如 `0-两手托天理三焦（八段锦）_reference_0`）
    if "(标签:" in label_line:
        end_idx = label_line.find(" (标签:")  # 提取标签部分前的信息
        key_info = label_line[:end_idx]  # 提取文件名关键部分
    
        # 提取前两个字符并提取其中的数字部分作为标签
        match = re.match(r"(\d+)", key_info)  # 提取关键部分前面数字的部分
        if match:
            label_value = int(match.group(1))  # 获取数字部分作为标签值
                
            # 提取 14 维向量数据
            data_values = list(map(float, data_line.split()))
                
            # 检查数据和标签范围
            if len(data_values) == 14 and all(0 <= v <= 1 for v in data_values):
                features.append(data_values)
                labels.append(label_value)
    
    i += 2  # 跳过标签行和数据行

# 转换为 NumPy 数组
features = np.array(features)
labels = np.array(labels)

# 检查数据范围
print("Feature values range:", np.min(features), np.max(features))
print("Label values range:", np.min(labels), np.max(labels))

# 构建 TensorFlow 数据集
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
train_size = int(0.7 * len(dataset))
train_dataset = dataset.take(train_size).batch(128)
val_dataset = dataset.skip(train_size).batch(128)

print(f"数据集构建完成：总样本数 {len(labels)}")
print(f"训练集样本数 {train_size}, 验证集样本数 {len(labels) - train_size}")


Feature values range: 4.286189e-21 0.99996305
Label values range: 0 14
数据集构建完成：总样本数 749
训练集样本数 524, 验证集样本数 225


In [2]:
import tensorflow as tf
def save_dataset(dataset, save_path):
    """
    使用最新方法保存 TensorFlow 数据集。
    
    Args:
        dataset: tf.data.Dataset 对象。
        save_path: 保存路径。
    """
    dataset.save(save_path)
    print(f"数据集已保存到: {save_path}")

# 保存数据集
save_path = "C:/data/result/saved_dataset_score_classify"
save_dataset(dataset, save_path)



数据集已保存到: C:/data/result/saved_dataset_score_classify


In [59]:
# 构建 TensorFlow 数据集
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
train_size = int( 0.9* len(dataset))
train_dataset = dataset.take(train_size).batch(128)
val_dataset = dataset.skip(train_size).batch(128)

In [51]:
from tensorflow.keras import layers, Model

# 定义 Wide & Deep 模型
def build_wide_and_deep_model(input_dim, num_classes=15):  # 修改为15个类别
    # 输入层
    inputs = layers.Input(shape=(input_dim,))

    # Wide 部分（线性模型）
    wide_output = layers.Dense(1, activation="linear", name="wide")(inputs)

    # Deep 部分
    deep = layers.Dense(128, activation="relu")(inputs)
    deep = layers.Dense(64, activation="relu")(deep)
    deep_output = layers.Dense(1, activation="linear", name="deep")(deep)

    # 合并 Wide 和 Deep 部分
    combined = layers.Add()([wide_output, deep_output])

    # 输出层，使用 softmax 激活函数，输出为 15 个类别的概率分布
    outputs = layers.Dense(num_classes, activation="softmax")(combined)

    # 构建模型
    model = Model(inputs=inputs, outputs=outputs, name="Wide_Deep_Model")
    return model

# 获取输入维度（14维）
input_dim = 14
model = build_wide_and_deep_model(input_dim)

# 查看模型结构
model.summary()

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",  # 使用稀疏类别交叉熵损失
              metrics=["accuracy"])


Model: "Wide_Deep_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 14)]         0           []                               
                                                                                                  
 dense_46 (Dense)               (None, 128)          1920        ['input_10[0][0]']               
                                                                                                  
 dense_47 (Dense)               (None, 64)           8256        ['dense_46[0][0]']               
                                                                                                  
 wide (Dense)                   (None, 1)            15          ['input_10[0][0]']               
                                                                                    

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",  # 使用稀疏类别交叉熵损失
              metrics=["accuracy"])
# 启动训练
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1500,  # 训练50轮
    verbose=1
)

Epoch 1/1500
5/5 [==============================] - 1s 46ms/step - loss: 2.7929 - accuracy: 0.0000e+00 - val_loss: 2.7978 - val_accuracy: 0.0000e+00
Epoch 2/1500
5/5 [==============================] - 0s 25ms/step - loss: 2.7693 - accuracy: 0.0000e+00 - val_loss: 2.8022 - val_accuracy: 0.0000e+00
Epoch 3/1500
5/5 [==============================] - 0s 23ms/step - loss: 2.7506 - accuracy: 0.0000e+00 - val_loss: 2.8062 - val_accuracy: 0.0000e+00
Epoch 4/1500
5/5 [==============================] - 0s 24ms/step - loss: 2.7333 - accuracy: 0.0000e+00 - val_loss: 2.8098 - val_accuracy: 0.0000e+00
Epoch 5/1500
5/5 [==============================] - 0s 18ms/step - loss: 2.7166 - accuracy: 0.0000e+00 - val_loss: 2.8140 - val_accuracy: 0.0000e+00
Epoch 6/1500
5/5 [==============================] - 0s 16ms/step - loss: 2.6993 - accuracy: 0.0017 - val_loss: 2.8184 - val_accuracy: 0.0000e+00
Epoch 7/1500
5/5 [==============================] - 0s 17ms/step - loss: 2.6810 - accuracy: 0.0835 - val_loss:

In [58]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.layers import BatchNormalization

def build_dnn_model_no_dropout(input_dim, num_classes=15):
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_dim,)),
        BatchNormalization(),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dense(num_classes, activation="softmax")
    ])
    return model

model = build_dnn_model_no_dropout(input_dim=14)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_57 (Dense)            (None, 256)               3840      
                                                                 
 batch_normalization_21 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dense_58 (Dense)            (None, 128)               32896     
                                                                 
 batch_normalization_22 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                                 
 dense_59 (Dense)            (None, 64)                8256      
                                                                 
 batch_normalization_23 (Bat  (None, 64)              

In [65]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=500,  # 训练50轮
    verbose=1
)

Epoch 1/500
6/6 [==============================] - 0s 36ms/step - loss: 0.0358 - accuracy: 1.0000 - val_loss: 2.9067 - val_accuracy: 0.3067
Epoch 2/500
6/6 [==============================] - 0s 28ms/step - loss: 0.0356 - accuracy: 1.0000 - val_loss: 2.9069 - val_accuracy: 0.3067
Epoch 3/500
6/6 [==============================] - 0s 32ms/step - loss: 0.0354 - accuracy: 1.0000 - val_loss: 2.9071 - val_accuracy: 0.3067
Epoch 4/500
6/6 [==============================] - 0s 29ms/step - loss: 0.0352 - accuracy: 1.0000 - val_loss: 2.9070 - val_accuracy: 0.3067
Epoch 5/500
6/6 [==============================] - 0s 23ms/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 2.9090 - val_accuracy: 0.3067
Epoch 6/500
6/6 [==============================] - 0s 23ms/step - loss: 0.0349 - accuracy: 1.0000 - val_loss: 2.9094 - val_accuracy: 0.3067
Epoch 7/500
6/6 [==============================] - 0s 24ms/step - loss: 0.0347 - accuracy: 1.0000 - val_loss: 2.9103 - val_accuracy: 0.3067
Epoch 8/500
6/6 [===

In [66]:
# 保存模型到指定路径
save_path = "c:/data/result/model/final_model.h5"
model.save(save_path)
print(f"模型已保存到 {save_path}")


模型已保存到 c:/data/result/model/final_model.h5
